In [1]:
import pandas as pd
import numpy as np
# StratifiedKFold
from sklearn.model_selection import StratifiedKFold
# CoxPHFitter
from sklearn.model_selection import train_test_split
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
# PCA
from sklearn.decomposition import PCA
from utils import SAkfold

In [2]:
# data biomarkers
df_bio = pd.read_csv('data/sustain_results_biomarkers_zscored.csv')
print(df_bio.columns)

Index(['Eid', 't1dm', 't2dm', 'other_dm', 'all_dm', 'Sex', 'Age',
       'Drinking_status', 'Smoking_status', 'Income', 'Education', 'BMI',
       'SBP', 'DBP', 'WBC', 'RBC', 'Hgb', 'Hct', 'MCV', 'MCH', 'MCHC', 'RDW',
       'PLT', 'PCT', 'MPV', 'PDW', 'LYM', 'MONO', 'NEUT', 'EOS', 'BASO',
       'NRBC', 'LYMP', 'MONOP', 'NEUTP', 'EOSP', 'BASOP', 'NRBCP', 'RETP',
       'RET', 'MRV', 'MSCV', 'IRF', 'HLS_ReticP', 'HLS_Retic',
       'Urine_creatinine', 'Urine_potassium', 'Urine_sodium', 'Albumin', 'ALP',
       'ALT', 'ApoA', 'ApoB', 'AST', 'DBIL', 'Urea', 'Calcium', 'CHOL',
       'Creatinine', 'CRP', 'CysC', 'Glucose', 'HbA1c', 'HDLc', 'IGF_1',
       'LDLc', 'Lpa', 'Phosphate', 'TBIL', 'Testosterone', 'TP', 'TG', 'Urate',
       'Vitamin_D', 'TyG', 'HbA1c_prec', 'propensity_score', 'ml_subtype',
       'ml_subtype_1', 'prob_ml_subtype', 'prob_ml_subtype_1', 'ml_stage',
       'ml_stage_1', 'prob_ml_stage', 'prob_ml_stage_1', 'Subtype', 'Stage',
       'eGFR'],
      dtype='object')


In [3]:
# biomarkers: HbA1c, BMI, HDLc, TyG, HLS_Retic, Glucose, RET, IRF, ApoA, ALT, TG, Urate, WBC, CHOL, CRP, LYM, LDLc, Vitamin_D
biomarkers =['HbA1c', 'BMI', 'HDLc', 'TyG', 'HLS_Retic', 'Glucose', 'RET', 'IRF', 'ApoA', 
             'ALT', 'TG', 'Urate', 'WBC', 'CHOL', 'CRP', 'LYM', 'LDLc', 'Vitamin_D']
diseases = [
    'Alzheimers_disease',
    'Anorexia_nervosa',
    'Anxiety_disorder',
    'Bipolar_disorder',
    'Depression_disorder',
    'Epilepsy',
    'Multiple_sclerosis',
    'Obsessive_compulsive_disorder',
    'Parkinsons_disease',
    'Schizophrenia',
    'Sleep_disorder',
    'Stroke',
    'hypertension', 
    'heart_failure', 
    'ischemic_heart_disease', 
    'cardiac_arrhythmias', 
    'acute_kidney_failure', 
    'chronic_kidney_disease',
    'glomerulus_nephritis',
    'obesity', 
    'retinopathy', 
    'asthma'
]

In [4]:
# Subtype
df_bio['Subtype1'] = np.where(((df_bio['Subtype'] == 'Subtype 1') & (df_bio['t2dm'] == 1)), 1, 0)
df_bio['Subtype2'] = np.where(((df_bio['Subtype'] == 'Subtype 2') & (df_bio['t2dm'] == 1)), 1, 0)
print(df_bio['Subtype1'].value_counts())
print(df_bio['Subtype2'].value_counts())

Subtype1
0    31208
1     9402
Name: count, dtype: int64
Subtype2
0    29707
1    10903
Name: count, dtype: int64


In [6]:
# prediction model 1, only use biomarkers for disease prediction
feats = biomarkers
res_baseline = SAkfold(diseases=diseases, df_bio=df_bio, kf=10, feats=feats)

SA for Alzheimers_disease
fold:  0
Alzheimers_disease : c-index on fold  0 :  0.5465607733336172
fold:  1
Alzheimers_disease : c-index on fold  1 :  0.5231597541755254
fold:  2
Alzheimers_disease : c-index on fold  2 :  0.621908865806991
fold:  3
Alzheimers_disease : c-index on fold  3 :  0.5923760110200516
fold:  4
Alzheimers_disease : c-index on fold  4 :  0.5347566501412655
fold:  5
Alzheimers_disease : c-index on fold  5 :  0.49904355344212686
fold:  6
Alzheimers_disease : c-index on fold  6 :  0.5782383250540015
fold:  7
Alzheimers_disease : c-index on fold  7 :  0.5499089872362823
fold:  8
Alzheimers_disease : c-index on fold  8 :  0.5629566315697782
fold:  9
Alzheimers_disease : c-index on fold  9 :  0.5667255491721969
SA for Anorexia_nervosa
fold:  0
Anorexia_nervosa : c-index on fold  0 :  0.8061281337047354
fold:  1
Anorexia_nervosa : c-index on fold  1 :  0.93265465935787
fold:  2
Anorexia_nervosa : c-index on fold  2 :  0.8643587532339534
fold:  3
Anorexia_nervosa : c-index

In [7]:
# mean, std of c-index
mean_c_index = np.mean(res_baseline, axis=1)
std_c_index = np.std(res_baseline, axis=1)
# to csv
df_res = pd.DataFrame({'disease': diseases, 'mean_c_index': mean_c_index, 'std_c_index': std_c_index})
df_res.to_csv('results/prediction/baseline_c_index.csv', index=False)

In [8]:
# prediction model 2, biomarkers and subtype 1 for disease prediction
# use the first N components as features for prediction
# feats = ['PC' + str(i) for i in range(N)]
feats = biomarkers + ['Subtype1']
res_s1 = SAkfold(diseases=diseases, df_bio=df_bio, kf=10, feats=feats)

SA for Alzheimers_disease
fold:  0
Alzheimers_disease : c-index on fold  0 :  0.5470717867370729
fold:  1
Alzheimers_disease : c-index on fold  1 :  0.530807391658469
fold:  2
Alzheimers_disease : c-index on fold  2 :  0.6084831334212304
fold:  3
Alzheimers_disease : c-index on fold  3 :  0.5983791519491448
fold:  4
Alzheimers_disease : c-index on fold  4 :  0.5329228636920944
fold:  5
Alzheimers_disease : c-index on fold  5 :  0.5058521560574949
fold:  6
Alzheimers_disease : c-index on fold  6 :  0.5851592297773014
fold:  7
Alzheimers_disease : c-index on fold  7 :  0.546049168823194
fold:  8
Alzheimers_disease : c-index on fold  8 :  0.5475223484064852
fold:  9
Alzheimers_disease : c-index on fold  9 :  0.5787354584082627
SA for Anorexia_nervosa
fold:  0
Anorexia_nervosa : c-index on fold  0 :  0.7649025069637883
fold:  1
Anorexia_nervosa : c-index on fold  1 :  0.9351996867658575
fold:  2
Anorexia_nervosa : c-index on fold  2 :  0.8680547000123198
fold:  3
Anorexia_nervosa : c-index

In [28]:
mean_c_index = np.mean(res_s1, axis=1)
std_c_index = np.std(res_s1, axis=1)
# to csv
df_res_s1 = pd.DataFrame({'disease': diseases, 'mean_c_index': mean_c_index, 'std_c_index': std_c_index})
df_res_s1.to_csv('results/prediction/subtype1_c_index.csv', index=False)

In [21]:
# prediction model 3, biomarkers and subtype 2 for disease prediction
# use the first N components as features for prediction
# feats = ['PC' + str(i) for i in range(N)]
feats = biomarkers + ['Subtype2']
res_s2 = SAkfold(diseases=diseases, df_bio=df_bio, kf=10, feats=feats)

SA for Alzheimers_disease
fold:  0
Alzheimers_disease : c-index on fold  0 :  0.5390978484206492
fold:  1
Alzheimers_disease : c-index on fold  1 :  0.5362813230518622
fold:  2
Alzheimers_disease : c-index on fold  2 :  0.6278258352895778
fold:  3
Alzheimers_disease : c-index on fold  3 :  0.62717278861976
fold:  4
Alzheimers_disease : c-index on fold  4 :  0.5503704888320273
fold:  5
Alzheimers_disease : c-index on fold  5 :  0.5151356316870205
fold:  6
Alzheimers_disease : c-index on fold  6 :  0.5637544819931335
fold:  7
Alzheimers_disease : c-index on fold  7 :  0.5798883722970306
fold:  8
Alzheimers_disease : c-index on fold  8 :  0.5912990929976808
fold:  9
Alzheimers_disease : c-index on fold  9 :  0.5668953005394911
SA for Anorexia_nervosa
fold:  0
Anorexia_nervosa : c-index on fold  0 :  0.8055710306406685
fold:  1
Anorexia_nervosa : c-index on fold  1 :  0.9436178543461238
fold:  2
Anorexia_nervosa : c-index on fold  2 :  0.8833312800295676
fold:  3
Anorexia_nervosa : c-index

In [29]:
# mean, std of c-index
mean_c_index = np.mean(res_s2, axis=1)
std_c_index = np.std(res_s2, axis=1)
# to csv
df_res_s2 = pd.DataFrame({'disease': diseases, 'mean_c_index': mean_c_index, 'std_c_index': std_c_index})
df_res_s2.to_csv('results/prediction/subtype2_c_index.csv', index=False)

In [31]:
# merge s1, s2, baseline
cols = df_res.columns.tolist()
df_res['group'] = 'Baseline'
df_res_s1['group'] = 'Subtype 1'
df_res_s2['group'] = 'Subtype 2'
df_res_all = pd.concat([df_res, df_res_s1, df_res_s2])
df_res_all = df_res_all[['group', 'disease', 'mean_c_index', 'std_c_index']]

In [32]:
# sort by disease, group
df_res_all = df_res_all.sort_values(by=['disease', 'group'])
# to csv
df_res_all.to_csv('results/prediction/all_c_index.csv', index=False)

In [25]:
# pvalues for subtype 1 and control
from scipy.stats import ttest_ind
pvalues_s1, pvalues_s2, pvalues_s12 = [], [], []
for disease in diseases:
    c_index_control = res_baseline[diseases.index(disease)]
    c_index_s1 = res_s1[diseases.index(disease)]
    c_index_s2 = res_s2[diseases.index(disease)]
    _, pvalue_s1c = ttest_ind(c_index_s1, c_index_control)
    _, pvalue_s2c = ttest_ind(c_index_s2, c_index_control)
    _, pvalue_s12 = ttest_ind(c_index_s1, c_index_s2)
    pvalues_s1.append(pvalue_s1c)
    pvalues_s2.append(pvalue_s2c)
    pvalues_s12.append(pvalue_s12)

In [26]:
# to csv
df_pvalues = pd.DataFrame({'disease': diseases, 'pvalue_s1': pvalues_s1, 'pvalue_s2': pvalues_s2, 'pvalue_s12': pvalues_s12})

In [27]:
df_pvalues.to_csv('results/prediction/pvalues_cindex.csv', index=False)

In [33]:
# save all fold results
diseases_rep = [disease for disease in diseases for _ in range(10)]
df_res_baseline_af = pd.DataFrame({'disease': diseases_rep, 'c_index': res_baseline.flatten(), 'group': 'Baseline'})
df_res_s1_af = pd.DataFrame({'disease': diseases_rep, 'c_index': res_s1.flatten(), 'group': 'Subtype 1'})
df_res_s2_af = pd.DataFrame({'disease': diseases_rep, 'c_index': res_s2.flatten(), 'group': 'Subtype 2'})
# merge
df_res_all_af = pd.concat([df_res_baseline_af, df_res_s1_af, df_res_s2_af], axis=0)
# to csv
df_res_all_af.to_csv('results/prediction/all_c_index_af.csv', index=False)

In [5]:
cph = CoxPHFitter()
df_sa = pd.read_csv('data/subtype1/survival_data_hypertension.csv')

In [6]:
# train test split
df_train, df_test = train_test_split(df_sa, test_size=0.2, random_state=42)

In [9]:
# drop column Eid and Stage
# df_train = df_train.drop(columns=['Eid', 'Stage'])
cph.fit(df_train, duration_col='time', event_col='hypertension')

<lifelines.CoxPHFitter: fitted with 19710 total observations, 11116 right-censored observations>

In [14]:
# predict survival at time 1-15 years
survival = cph.predict_survival_function(df_test[['Subtype']], times=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])